In [1]:
### Gemini 2D using Pycox chapter 3=====================

#================== Import packages===========================================
import numpy as np
import statistics
import matplotlib.pyplot
import pathlib
import nibabel as nib
import pandas as pd
import os
from torch.utils.data import Dataset, DataLoader 
import torchvision.transforms as transforms
from torch.autograd import Variable
from skimage.transform import resize
import scipy.io as sio

#=============For preprocessing===============================================
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper

#=====================For network buidling=====================================
import torch
from torch import nn
import torch.nn.functional as F
import torchtuples as tt

#============PyCox survival packages==========================================
from pycox.datasets import metabric
from pycox.models import LogisticHazard
from pycox.models import CoxPH
# from pycox.models import PMF
from pycox.models.loss import NLLLogistiHazardLoss
from pycox.evaluation import EvalSurv

np.random.seed(12345)
_ = torch.manual_seed(1235)

# Pred

In [12]:
#========Splitting ========================
from sklearn.model_selection import KFold
# ===========================Datasset=========================================
root_dir = pathlib.Path.cwd()

data = pd.read_csv( 'E:/papers/CT2PET/Survival/PROSPECT_PRED_68.csv')
data0 = pd.read_csv('E:/papers/CT2PET/Survival/PROSPECT_Pred_MTV_TLG_pall.csv')
data.sort_values('MRN')
data0.sort_values('MRN')
data['MTV_1.5'] = data0['MTV_1.5'] 
data['TLG_1.5'] = data0['TLG_1.5'] 
data['MTV_2.5_v2'] = data0['MTV_2.5'] 

data1 = pd.DataFrame.copy(data)

data1['ID'] = data1['MRN']
data1 = data1.drop(columns="MRN")

data1.head(5)

data_vol = pd.DataFrame.copy(data1)

In [13]:
xyz = np.array(data0['Tumor_vol'].tolist())/np.array(data0['Tumor_pix'].tolist())
print(np.min(xyz),np.max(xyz))
columns_data1 = [ 'MTV_0.5', 'TLG_0.5', 'MTV_1.5', 'TLG_1.5', 'MTV_2.5', 'TLG_2.5','SUV_mean']

for col in columns_data1:
    data_vol[col] = data1[col]*data1['Tumor_vol']/data1['Tumor_pix']
data_vol.head()  

0.24414062504491296 0.9313212107208872


,MTV_0.5,TLG_0.5,MTV_1.0,TLG_1.0,MTV_1.0_vol,TLG_1.0_vol,MTV_2.5,TLG_2.5,MTV_2.5_vol,TLG_2.5_vol,SUV_max,SUV_mean,Tumor_pix,Tumor_vol,OS_events,OS,MTV_1.5,TLG_1.5,MTV_2.5_v2,ID
0,15601.492913,62781.052004,14435,65580.148740,13443.62167,61076.183500,9490.163132,54966.658022,9490.163134,54966.658030,7.000000,3.743337,16774,15621.981980,1,1.509589,11337.904413,58472.820454,10190,688647
1,19494.533540,28541.668207,27649,49536.465280,13184.07059,23620.827330,1996.517182,6249.281412,1996.517181,6249.281412,5.159066,0.680637,42375,20205.974580,1,1.367123,7330.894471,16416.037489,4187,674798
2,1075.675998,1094.903153,504,680.160169,469.38589,633.447592,0.000000,0.000000,0.000000,0.000000,2.151847,0.921277,1217,1133.417913,0,2.739726,128.522327,219.584611,0,624686
3,4039.973259,3513.660855,3190,4141.912886,1108.88958,1439.788100,4.518986,11.503441,4.518986,11.503441,2.593721,0.251258,16680,5798.206329,0,2.972603,180.759430,328.673429,13,617336
4,18124.718738,34556.082046,43069,87675.411100,16253.97725,33088.164050,2910.839038,11318.849392,2910.839038,11318.849400,7.000000,0.716555,48289,18223.973330,1,0.421918,10962.537582,26310.010559,7713,391043


In [14]:
columns_data1 = [ 'MTV_0.5', 'TLG_0.5', 'MTV_1.5', 'TLG_1.5', 'MTV_2.5', 'TLG_2.5','SUV_mean']
# data1['MTV_2.5'].replace(to_replace=0, value = 1e-3)
# data1['TLG_2.5'].replace(to_replace=0, value = 1e-3)

def log_then_zscore(l, z_score = False):
    epsilon = 1
    l = l.tolist()
    L_np = np.array(l).astype(np.float32) # if int then next line does not work!!!   L_np[L_np==0] = epsilon 
    #print(np.sum(L_np==0), np.sum(L_np<0) ,L_np.dtype)
    L_np[L_np==0] = epsilon
    l_new = np.log(L_np)
    if z_score:
        l_new = (l_new - np.mean(l_new))/np.std(l_new)
    return l_new

print("Befor3")
for col in columns_data1:
    l = data_vol[col].tolist()
    L_np = np.array(l)
    print(col, np.sum(L_np==0), np.sum(L_np<0) )


for col in columns_data1:
    #print(col)
    data_vol[col] = log_then_zscore(data_vol[col],True)

print()

print("AFter")
for col in columns_data1:
    l = data_vol[col].tolist()
    L_np = np.array(l)
    print(col, np.sum(L_np==0) ) # can be negative


Befor3
MTV_0.5 0 0
TLG_0.5 0 0
MTV_1.5 1 0
TLG_1.5 1 0
MTV_2.5 5 0
TLG_2.5 5 0
SUV_mean 0 0

AFter
MTV_0.5 0
TLG_0.5 0
MTV_1.5 0
TLG_1.5 0
MTV_2.5 0
TLG_2.5 0
SUV_mean 0


In [15]:
from lifelines import CoxPHFitter
df4 = pd.DataFrame(columns=['OS','Event'])

cut_off = np.quantile( data_vol["Tumor_vol"], 0.0)
criter  = data_vol["Tumor_vol"]>cut_off


df4['OS']        = data_vol['OS'][criter].astype(np.float32)
df4['Event']     = data_vol['OS_events'][criter].astype(bool)
df4['Tumor_vol'] = data_vol['Tumor_vol'][criter].astype(np.float32)

cph = CoxPHFitter()
cph.fit(df4, 'OS', 'Event')  #, strata=["strata"]
cph.print_summary()
print('Tumor_vol: ',round(cph.concordance_index_,4))

'''D={'MTV_0.5': x,
      'TLG_0.5'  x,
      'MTV_1.0': x,
      'TLG_1.0': x,
      'MTV_2.5': x,
      'TLG_2.5': x,
      'Tumor_vol':x,
      'SUV_max':  x,
      'SUV_mean': x 
  }'''

####################################################################
df5 = pd.DataFrame(columns=['OS','Event'])
df5['OS']        = data_vol['OS'][criter].astype(np.float32)
df5['Event']     = data_vol['OS_events'][criter].astype(bool)
df5['Tumor_vol'] = data_vol['MTV_0.5'][criter].astype(np.float32)

cph = CoxPHFitter()
cph.fit(df5, 'OS', 'Event')  #, strata=["strata"]
cph.print_summary()
print('MTV_0.5: ',round(cph.concordance_index_,4))

####################################################################
df5 = pd.DataFrame(columns=['OS','Event'])
df5['OS']        = data_vol['OS'][criter].astype(np.float32)
df5['Event']     = data_vol['OS_events'][criter].astype(bool)
df5['Tumor_vol'] = data_vol['MTV_1.5'][criter].astype(np.float32)

cph = CoxPHFitter()
cph.fit(df5, 'OS', 'Event')  #, strata=["strata"]
cph.print_summary()
print('MTV_1.5: ',round(cph.concordance_index_,4))
####################################################################
df5 = pd.DataFrame(columns=['OS','Event'])
df5['OS']        = data_vol['OS'][criter].astype(np.float32)
df5['Event']     = data_vol['OS_events'][criter].astype(bool)
df5['Tumor_vol'] = data_vol['MTV_2.5'][criter].astype(np.float32)

cph = CoxPHFitter()
cph.fit(df5, 'OS', 'Event')  #, strata=["strata"]
cph.print_summary()
print('MTV_2.5: ',round(cph.concordance_index_,4))
#####################################################################
df5 = pd.DataFrame(columns=['OS','Event'])
df5['OS']        = data_vol['OS'][criter].astype(np.float32)
df5['Event']     = data_vol['OS_events'][criter].astype(bool)
df5['Tumor_vol'] = data_vol['TLG_0.5'][criter].astype(np.float32)

cph = CoxPHFitter()
cph.fit(df5, 'OS', 'Event')  #, strata=["strata"]
cph.print_summary()
print('TLG_0.5',round(cph.concordance_index_,4))
#####################################################################
df5 = pd.DataFrame(columns=['OS','Event'])
df5['OS']        = data_vol['OS'][criter].astype(np.float32)
df5['Event']     = data_vol['OS_events'][criter].astype(bool)
df5['Tumor_vol'] = data_vol['TLG_1.5'][criter].astype(np.float32)

cph = CoxPHFitter()
cph.fit(df5, 'OS', 'Event')  #, strata=["strata"]
cph.print_summary()
print('TLG_1.5: ',round(cph.concordance_index_,4))
#####################################################################
df5 = pd.DataFrame(columns=['OS','Event'])
df5['OS']        = data_vol['OS'][criter].astype(np.float32)
df5['Event']     = data_vol['OS_events'][criter].astype(bool)
df5['Tumor_vol'] = data_vol['TLG_2.5'][criter].astype(np.float32)

cph = CoxPHFitter()
cph.fit(df5, 'OS', 'Event')  #, strata=["strata"]
cph.print_summary()
print('TLG_2.5: ',round(cph.concordance_index_,4))
#####################################################################
df5 = pd.DataFrame(columns=['OS','Event'])
df5['OS']        = data_vol['OS'][criter].astype(np.float32)
df5['Event']     = data_vol['OS_events'][criter].astype(bool)
df5['Tumor_vol'] = data_vol['SUV_max'][criter].astype(np.float32)

cph = CoxPHFitter()
cph.fit(df5, 'OS', 'Event')  #, strata=["strata"]
cph.print_summary()
print('SUV_max: ',round(cph.concordance_index_,4))
#####################################################################
df5 = pd.DataFrame(columns=['OS','Event'])
df5['OS']        = data_vol['OS'][criter].astype(np.float32)
df5['Event']     = data_vol['OS_events'][criter].astype(bool)
df5['Tumor_vol'] = data_vol['SUV_mean'][criter].astype(np.float32)

cph = CoxPHFitter()
cph.fit(df5, 'OS', 'Event')  #, strata=["strata"]
cph.print_summary()
print('SUV_mean: ',round(cph.concordance_index_,4))

<lifelines.CoxPHFitter: fitted with 67 total observations, 38 right-censored observations>
             duration col = 'OS'
                event col = 'Event'
      baseline estimation = breslow
   number of observations = 67
number of events observed = 29
   partial log-likelihood = -96.97
         time fit was run = 2022-08-09 04:53:25 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
Tumor_vol   0.00       1.00       0.00             0.00             0.00                 1.00                 1.00

             z      p   -log2(p)
covariate                       
Tumor_vol 3.98 <0.005      13.83
---
Concordance = 0.66
Partial AIC = 195.94
log-likelihood ratio test = 11.93 on 1 df
-log2(p) of ll-ratio test = 10.82

Tumor_vol:  0.6561


<lifelines.CoxPHFitter: fitted with 67 total observations, 38 right-censored observations>
             duration col = 'OS'
                event col = 'Event'
      baseline estimation = breslow
   number of observations = 67
number of events observed = 29
   partial log-likelihood = -97.47
         time fit was run = 2022-08-09 04:53:25 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
Tumor_vol   0.75       2.12       0.24             0.28             1.22                 1.32                 3.40

             z      p   -log2(p)
covariate                       
Tumor_vol 3.12 <0.005       9.12
---
Concordance = 0.66
Partial AIC = 196.93
log-likelihood ratio test = 10.94 on 1 df
-log2(p) of ll-ratio test = 10.06

MTV_0.5:  0.657


<lifelines.CoxPHFitter: fitted with 67 total observations, 38 right-censored observations>
             duration col = 'OS'
                event col = 'Event'
      baseline estimation = breslow
   number of observations = 67
number of events observed = 29
   partial log-likelihood = -100.54
         time fit was run = 2022-08-09 04:53:25 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
Tumor_vol   0.57       1.77       0.28             0.03             1.12                 1.03                 3.06

             z    p   -log2(p)
covariate                     
Tumor_vol 2.06 0.04       4.67
---
Concordance = 0.64
Partial AIC = 203.08
log-likelihood ratio test = 4.80 on 1 df
-log2(p) of ll-ratio test = 5.13

...

TLG_2.5:  0.5776


<lifelines.CoxPHFitter: fitted with 67 total observations, 38 right-censored observations>
             duration col = 'OS'
                event col = 'Event'
      baseline estimation = breslow
   number of observations = 67
number of events observed = 29
   partial log-likelihood = -102.55
         time fit was run = 2022-08-09 04:53:26 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
Tumor_vol   0.10       1.10       0.12            -0.13             0.33                 0.88                 1.39

             z    p   -log2(p)
covariate                     
Tumor_vol 0.85 0.40       1.34
---
Concordance = 0.52
Partial AIC = 207.11
log-likelihood ratio test = 0.77 on 1 df
-log2(p) of ll-ratio test = 1.39

SUV_max:  0.5179


<lifelines.CoxPHFitter: fitted with 67 total observations, 38 right-censored observations>
             duration col = 'OS'
                event col = 'Event'
      baseline estimation = breslow
   number of observations = 67
number of events observed = 29
   partial log-likelihood = -102.28
         time fit was run = 2022-08-09 04:53:26 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
Tumor_vol  -0.21       0.81       0.18            -0.57             0.15                 0.57                 1.16

              z    p   -log2(p)
covariate                      
Tumor_vol -1.15 0.25       2.00
---
Concordance = 0.62
Partial AIC = 206.56
log-likelihood ratio test = 1.31 on 1 df
-log2(p) of ll-ratio test = 1.99

SUV_mean:  0.6177


In [7]:
df5 = pd.DataFrame(columns=['OS','Event'])
df5['OS']        = data_vol['OS'][criter].astype(np.float32)
df5['Event']     = data_vol['OS_events'][criter].astype(bool)
df5['Tumor_pix'] = data_vol['Tumor_pix'][criter].astype(np.float32)

cph = CoxPHFitter()
cph.fit(df5, 'OS', 'Event')  #, strata=["strata"]
cph.print_summary()
print('Tumor_pix: ',round(cph.concordance_index_,4))

<lifelines.CoxPHFitter: fitted with 67 total observations, 38 right-censored observations>
             duration col = 'OS'
                event col = 'Event'
      baseline estimation = breslow
   number of observations = 67
number of events observed = 29
   partial log-likelihood = -97.82
         time fit was run = 2022-08-09 04:43:55 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
Tumor_pix   0.00       1.00       0.00             0.00             0.00                 1.00                 1.00

             z      p   -log2(p)
covariate                       
Tumor_pix 3.77 <0.005      12.59
---
Concordance = 0.69
Partial AIC = 197.65
log-likelihood ratio test = 10.23 on 1 df
-log2(p) of ll-ratio test = 9.50

Tumor_pix:  0.6945


# TRuth

In [8]:
#========Splitting ========================
from sklearn.model_selection import KFold
# ===========================Datasset=========================================
#root_dir = pathlib.Path.cwd()

data = pd.read_csv( 'E:/papers/CT2PET/Survival/PROSPECT_TRUTH_68.csv')
data0 = pd.read_csv('E:/papers/CT2PET/Survival/PROSPECT_True_MTV_TLG_pall.csv')
data = data.sort_values('MRN')
data0 = data0.sort_values('MRN')

data['MTV_1.5'] = data0['MTV_1.5'].tolist()
data['TLG_1.5'] = data0['TLG_1.5'].tolist()
data['MTV_2.5_v2'] = data0['MTV_2.5'].tolist()

data2 = pd.DataFrame.copy(data)

data2['ID'] = data2['MRN']
#data1 = data1.drop(columns="MRN")


data_vol_t = pd.DataFrame.copy(data2)
data1.head(5)

,MTV_0.5,TLG_0.5,MTV_1.0,TLG_1.0,MTV_1.0_vol,TLG_1.0_vol,MTV_2.5,TLG_2.5,MTV_2.5_vol,TLG_2.5_vol,SUV_max,SUV_mean,Tumor_pix,Tumor_vol,OS_events,OS,MTV_1.5,TLG_1.5,MTV_2.5_v2,ID
0,16752,67410.740050,14435,65580.148740,13443.62167,61076.183500,10190,59020.086110,9490.163134,54966.658030,7.000000,4.019384,16774,15621.981980,1,1.509589,12174,62784.804870,10190,688647
1,40883,59856.216560,27649,49536.465280,13184.07059,23620.827330,4187,13105.693010,1996.517181,6249.281412,5.159066,1.427400,42375,20205.974580,1,1.367123,15374,34426.925850,4187,674798
2,1155,1175.645031,504,680.160169,469.38589,633.447592,0,0.000000,0.000000,0.000000,2.151847,0.989215,1217,1133.417913,0,2.739726,138,235.777526,0,624686
3,11622,10107.929890,3190,4141.912886,1108.88958,1439.788100,13,33.092543,4.518986,11.503441,2.593721,0.722806,16680,5798.206329,0,2.972603,520,945.511852,13,617336
4,48026,91565.028970,43069,87675.411100,16253.97725,33088.164050,7713,29992.137740,2910.839038,11318.849400,7.000000,1.898693,48289,18223.973330,1,0.421918,29048,69714.988980,7713,391043


In [9]:
# xyz = np.array(data0['Tumor_vol'].tolist())/np.array(data0['Tumor_pix'].tolist())
# print(np.min(xyz),np.max(xyz))
columns_data1 = [ 'MTV_0.5', 'TLG_0.5', 'MTV_1.5', 'TLG_1.5', 'MTV_2.5', 'TLG_2.5','SUV_mean']

for col in columns_data1:
    data_vol_t[col] = data2[col]*data2['Tumor_vol']/data2['Tumor_pix']
data_vol_t.head()  

,MRN,MTV_0.5,TLG_0.5,MTV_1.0,TLG_1.0,MTV_1.0_vol,TLG_1.0_vol,MTV_2.5,TLG_2.5,MTV_2.5_vol,...,SUV_max,SUV_mean,Tumor_pix,Tumor_vol,OS_events,OS,MTV_1.5,TLG_1.5,MTV_2.5_v2,ID
18,248198,10496.921360,33928.182999,11271,36430.16247,10496.921360,33928.183010,6482.926944,25746.456402,6482.926945,...,7.905644,3.010219,11271,10496.921360,1,4.109589,10463.393796,33883.732611,6961,248198
11,271505,76779.983230,113423.729228,68235,108575.25380,63548.702790,101118.436800,3587.449303,10779.987793,3587.449302,...,4.286385,1.375800,82442,76779.983230,0,5.446575,30496.113035,60814.138019,3852,271505
58,272245,15725.027560,89612.144442,45237,257791.89020,15725.027560,89612.144460,15725.027560,89612.144442,15725.027560,...,12.553775,1.980948,45237,15725.027560,1,1.695890,15725.027560,89612.144442,45237,272245
27,281227,15943.287800,16388.886405,8558,10176.03079,7970.246918,9477.153308,0.931321,2.372257,0.931321,...,2.547195,0.957351,17119,15943.287800,0,1.964384,496.394205,835.839764,1,281227
5,294719,8373.509002,120769.204608,8991,129675.13600,8373.509002,120769.204700,8373.509002,120769.204608,8373.509002,...,25.748968,13.432233,8991,8373.509002,1,5.175342,8373.509002,120769.204608,8991,294719


In [10]:
columns_data1 = [ 'MTV_0.5', 'TLG_0.5', 'MTV_1.5', 'TLG_1.5', 'MTV_2.5', 'TLG_2.5', 'SUV_mean']


def log_then_zscore(l, z_score = False):
    epsilon = 1
    l = l.tolist()
    L_np = np.array(l).astype(np.float32) # if int then next line does not work!!!   L_np[L_np==0] = epsilon 
    #print(np.sum(L_np==0), np.sum(L_np<0) ,L_np.dtype)
    L_np[L_np==0] = epsilon
    l_new = np.log(L_np)
    if z_score:
        l_new = (l_new - np.mean(l_new))/np.std(l_new)
    return l_new

print("Befor3")
for col in columns_data1:
    l = data_vol_t[col].tolist()
    L_np = np.array(l)
    print(col, np.sum(L_np==0), np.sum(L_np<0) )


for col in columns_data1:
    #print(col)
    data_vol_t[col] = log_then_zscore(data_vol_t[col],True)

print()

print("AFter")
for col in columns_data1:
    l = data_vol_t[col].tolist()
    L_np = np.array(l)
    print(col, np.sum(L_np==0) ) # can be negative


Befor3
MTV_0.5 0 0
TLG_0.5 0 0
MTV_1.5 0 0
TLG_1.5 0 0
MTV_2.5 1 0
TLG_2.5 1 0
SUV_mean 0 0

AFter
MTV_0.5 0
TLG_0.5 0
MTV_1.5 0
TLG_1.5 0
MTV_2.5 0
TLG_2.5 0
SUV_mean 0


In [11]:
from lifelines import CoxPHFitter
df4 = pd.DataFrame(columns=['OS','Event'])

cut_off = np.quantile( data_vol_t["Tumor_vol"], 0.00)
criter  = data_vol_t["Tumor_vol"]>cut_off


df4['OS']        = data_vol_t['OS'][criter].astype(np.float32)
df4['Event']     = data_vol_t['OS_events'][criter].astype(bool)
df4['Tumor_vol'] = data_vol_t['Tumor_vol'][criter].astype(np.float32)

cph = CoxPHFitter()
cph.fit(df4, 'OS', 'Event')  #, strata=["strata"]
cph.print_summary()
print('Tumor_vol: ',cph.concordance_index_)

'''D={'MTV_0.5': x,
      'TLG_0.5'  x,
      'MTV_1.0': x,
      'TLG_1.0': x,
      'MTV_2.5': x,
      'TLG_2.5': x,
      'Tumor_vol':x,
      'SUV_max':  x,
      'SUV_mean': x 
  }'''

####################################################################
df5 = pd.DataFrame(columns=['OS','Event'])
df5['OS']        = data_vol_t['OS'][criter].astype(np.float32)
df5['Event']     = data_vol_t['OS_events'][criter].astype(bool)
df5['Tumor_vol'] = data_vol_t['MTV_0.5'][criter].astype(np.float32)

cph = CoxPHFitter()
cph.fit(df5, 'OS', 'Event')  #, strata=["strata"]
cph.print_summary()
print('MTV_0.5: ',cph.concordance_index_)

####################################################################
df5 = pd.DataFrame(columns=['OS','Event'])
df5['OS']        = data_vol_t['OS'][criter].astype(np.float32)
df5['Event']     = data_vol_t['OS_events'][criter].astype(bool)
df5['Tumor_vol'] = data_vol_t['MTV_1.5'][criter].astype(np.float32)

cph = CoxPHFitter()
cph.fit(df5, 'OS', 'Event')  #, strata=["strata"]
cph.print_summary()
print('MTV_1.5: ',cph.concordance_index_)
####################################################################
df5 = pd.DataFrame(columns=['OS','Event'])
df5['OS']        = data_vol_t['OS'][criter].astype(np.float32)
df5['Event']     = data_vol_t['OS_events'][criter].astype(bool)
df5['Tumor_vol'] = data_vol_t['MTV_2.5'][criter].astype(np.float32)

cph = CoxPHFitter()
cph.fit(df5, 'OS', 'Event')  #, strata=["strata"]
cph.print_summary()
print('MTV_2.5: ',cph.concordance_index_)
#####################################################################
df5 = pd.DataFrame(columns=['OS','Event'])
df5['OS']        = data_vol_t['OS'][criter].astype(np.float32)
df5['Event']     = data_vol_t['OS_events'][criter].astype(bool)
df5['Tumor_vol'] = data_vol_t['TLG_0.5'][criter].astype(np.float32)

cph = CoxPHFitter()
cph.fit(df5, 'OS', 'Event')  #, strata=["strata"]
cph.print_summary()
print('TLG_0.5',cph.concordance_index_)
#####################################################################
df5 = pd.DataFrame(columns=['OS','Event'])
df5['OS']        = data_vol_t['OS'][criter].astype(np.float32)
df5['Event']     = data_vol_t['OS_events'][criter].astype(bool)
df5['Tumor_vol'] = data_vol_t['TLG_1.5'][criter].astype(np.float32)

cph = CoxPHFitter()
cph.fit(df5, 'OS', 'Event')  #, strata=["strata"]
cph.print_summary()
print('TLG_1.5: ',cph.concordance_index_)
#####################################################################
df5 = pd.DataFrame(columns=['OS','Event'])
df5['OS']        = data_vol_t['OS'][criter].astype(np.float32)
df5['Event']     = data_vol_t['OS_events'][criter].astype(bool)
df5['Tumor_vol'] = data_vol_t['TLG_2.5'][criter].astype(np.float32)

cph = CoxPHFitter()
cph.fit(df5, 'OS', 'Event')  #, strata=["strata"]
cph.print_summary()
print('TLG_2.5: ',cph.concordance_index_)
#####################################################################
df5 = pd.DataFrame(columns=['OS','Event'])
df5['OS']        = data_vol_t['OS'][criter].astype(np.float32)
df5['Event']     = data_vol_t['OS_events'][criter].astype(bool)
df5['Tumor_vol'] = data_vol_t['SUV_max'][criter].astype(np.float32)

cph = CoxPHFitter()
cph.fit(df5, 'OS', 'Event')  #, strata=["strata"]
cph.print_summary()
print('SUV_max: ',cph.concordance_index_)
#####################################################################
df5 = pd.DataFrame(columns=['OS','Event'])
df5['OS']        = data_vol_t['OS'][criter].astype(np.float32)
df5['Event']     = data_vol_t['OS_events'][criter].astype(bool)
df5['Tumor_vol'] = data_vol_t['SUV_mean'][criter].astype(np.float32)

cph = CoxPHFitter()
cph.fit(df5, 'OS', 'Event')  #, strata=["strata"]
cph.print_summary()
print('SUV_mean: ',cph.concordance_index_)
#####################################################################
df5 = pd.DataFrame(columns=['OS','Event'])
df5['OS']        = data_vol_t['OS'][criter].astype(np.float32)
df5['Event']     = data_vol_t['OS_events'][criter].astype(bool)
df5['Tumor_vol'] = data_vol_t['Tumor_vol'][criter].astype(np.float32)

cph = CoxPHFitter()
cph.fit(df5, 'OS', 'Event')  #, strata=["strata"]
cph.print_summary()
print('Tumor_vol: ',cph.concordance_index_)

<lifelines.CoxPHFitter: fitted with 67 total observations, 39 right-censored observations>
             duration col = 'OS'
                event col = 'Event'
      baseline estimation = breslow
   number of observations = 67
number of events observed = 28
   partial log-likelihood = -96.22
         time fit was run = 2022-08-09 04:53:18 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
Tumor_vol   0.00       1.00       0.00             0.00             0.00                 1.00                 1.00

             z      p   -log2(p)
covariate                       
Tumor_vol 2.89 <0.005       8.01
---
Concordance = 0.67
Partial AIC = 194.43
log-likelihood ratio test = 5.97 on 1 df
-log2(p) of ll-ratio test = 6.10

Tumor_vol:  0.6737400530503979


<lifelines.CoxPHFitter: fitted with 67 total observations, 39 right-censored observations>
             duration col = 'OS'
                event col = 'Event'
      baseline estimation = breslow
   number of observations = 67
number of events observed = 28
   partial log-likelihood = -94.04
         time fit was run = 2022-08-09 04:53:18 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
Tumor_vol   0.68       1.98       0.21             0.26             1.10                 1.30                 3.01

             z      p   -log2(p)
covariate                       
Tumor_vol 3.19 <0.005       9.48
---
Concordance = 0.67
Partial AIC = 190.09
log-likelihood ratio test = 10.31 on 1 df
-log2(p) of ll-ratio test = 9.56

MTV_0.5:  0.6737400530503979


<lifelines.CoxPHFitter: fitted with 67 total observations, 39 right-censored observations>
             duration col = 'OS'
                event col = 'Event'
      baseline estimation = breslow
   number of observations = 67
number of events observed = 28
   partial log-likelihood = -91.86
         time fit was run = 2022-08-09 04:53:18 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
Tumor_vol   0.94       2.57       0.26             0.43             1.46                 1.53                 4.30

             z      p   -log2(p)
covariate                       
Tumor_vol 3.58 <0.005      11.52
---
Concordance = 0.69
Partial AIC = 185.72
log-likelihood ratio test = 14.68 on 1 df
-log2(p) of ll-ratio test = 12.94

...

SUV_max:  0.579133510167993


<lifelines.CoxPHFitter: fitted with 67 total observations, 39 right-censored observations>
             duration col = 'OS'
                event col = 'Event'
      baseline estimation = breslow
   number of observations = 67
number of events observed = 28
   partial log-likelihood = -99.20
         time fit was run = 2022-08-09 04:53:19 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
Tumor_vol   0.00       1.00       0.19            -0.37             0.38                 0.69                 1.46

             z    p   -log2(p)
covariate                     
Tumor_vol 0.03 0.98       0.03
---
Concordance = 0.47
Partial AIC = 200.40
log-likelihood ratio test = 0.00 on 1 df
-log2(p) of ll-ratio test = 0.03

SUV_mean:  0.47038019451812557


<lifelines.CoxPHFitter: fitted with 67 total observations, 39 right-censored observations>
             duration col = 'OS'
                event col = 'Event'
      baseline estimation = breslow
   number of observations = 67
number of events observed = 28
   partial log-likelihood = -96.22
         time fit was run = 2022-08-09 04:53:19 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
Tumor_vol   0.00       1.00       0.00             0.00             0.00                 1.00                 1.00

             z      p   -log2(p)
covariate                       
Tumor_vol 2.89 <0.005       8.01
---
Concordance = 0.67
Partial AIC = 194.43
log-likelihood ratio test = 5.97 on 1 df
-log2(p) of ll-ratio test = 6.10

Tumor_vol:  0.6737400530503979
